In [4]:
import pandas as pd
import numpy as np

In [9]:
train_dataset = pd.read_csv('data/train.csv')
train_dataset['date_parsed'] = pd.to_datetime(train_dataset['date'])
train_dataset.head()

,id,date,store_nbr,family,sales,onpromotion,date_parsed
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,2013-01-01
1,1,2013-01-01,1,BABY CARE,0.0,0,2013-01-01
2,2,2013-01-01,1,BEAUTY,0.0,0,2013-01-01
3,3,2013-01-01,1,BEVERAGES,0.0,0,2013-01-01
4,4,2013-01-01,1,BOOKS,0.0,0,2013-01-01


In [10]:
train_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000888 entries, 0 to 3000887
Data columns (total 7 columns):
 #   Column       Dtype         
---  ------       -----         
 0   id           int64         
 1   date         object        
 2   store_nbr    int64         
 3   family       object        
 4   sales        float64       
 5   onpromotion  int64         
 6   date_parsed  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(3), object(2)
memory usage: 160.3+ MB


In [12]:
train_dataset.nunique()

id             3000888
date              1684
store_nbr           54
family              33
sales           379610
onpromotion        362
date_parsed       1684
dtype: int64

In [15]:
test_dataset = pd.read_csv('data/test.csv')
test_dataset['date_parsed'] = pd.to_datetime(test_dataset['date'])
test_dataset.head()

,id,date,store_nbr,family,onpromotion,date_parsed
0,3000888,2017-08-16,1,AUTOMOTIVE,0,2017-08-16
1,3000889,2017-08-16,1,BABY CARE,0,2017-08-16
2,3000890,2017-08-16,1,BEAUTY,2,2017-08-16
3,3000891,2017-08-16,1,BEVERAGES,20,2017-08-16
4,3000892,2017-08-16,1,BOOKS,0,2017-08-16


In [16]:
test_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28512 entries, 0 to 28511
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   id           28512 non-null  int64         
 1   date         28512 non-null  object        
 2   store_nbr    28512 non-null  int64         
 3   family       28512 non-null  object        
 4   onpromotion  28512 non-null  int64         
 5   date_parsed  28512 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(3), object(2)
memory usage: 1.3+ MB


In [17]:
test_dataset.nunique()

id             28512
date              16
store_nbr         54
family            33
onpromotion      212
date_parsed       16
dtype: int64

In [37]:
def resumir_df(df: pd.DataFrame, name:str):
    print(f'Formato do DataFrame "{name}": {df.shape}')
    df_resumo = pd.DataFrame(df.dtypes, columns=['Tipo'])
    df_resumo['Valores faltantes'] = df.isnull().sum().values
    df_resumo['Percentual de valores faltantes'] = (df.isnull().sum().values / len(df) * 100)
    df_resumo['Valores únicos'] = df.nunique().values
    desc = pd.DataFrame(df.describe(include='all').transpose())
    df_resumo['Min'] = desc['min'].values
    df_resumo['Máx'] = desc['max'].values
    return df_resumo.sort_index()

In [38]:
resumir_df(train_dataset, "Train")

Formato do DataFrame "Train": (3000888, 7)
Index(['id', 'date', 'store_nbr', 'family', 'sales', 'onpromotion',
       'date_parsed'],
      dtype='object')


,Tipo,Valores faltantes,Percentual de valores faltantes,Valores únicos,Min,Máx
date,object,0,0.0,1684,NaN,NaN
date_parsed,datetime64[ns],0,0.0,1684,2013-01-01 00:00:00,2017-08-15 00:00:00
family,object,0,0.0,33,NaN,NaN
id,int64,0,0.0,3000888,0.0,3000887.0
onpromotion,int64,0,0.0,362,0.0,741.0
sales,float64,0,0.0,379610,0.0,124717.0
store_nbr,int64,0,0.0,54,1.0,54.0


In [25]:
resumir_df(test_dataset, "Test")

Formato do DataFrame "Test": (28512, 6)


,Tipo,Valores faltantes,Percentual de valores faltantes,Valores únicos,Min,Máx
date,object,0,0.0,16,NaN,NaN
date_parsed,datetime64[ns],0,0.0,16,2017-08-16 00:00:00,2017-08-31 00:00:00
family,object,0,0.0,33,NaN,NaN
store_nbr,int64,0,0.0,54,1.0,54.0
onpromotion,int64,0,0.0,212,0.0,646.0
id,int64,0,0.0,28512,3000888.0,3029399.0
